# 3) Analyze and evaluate optimization output - EXPERIMENTAL

This final notebook uses the `runs.pkl` file created in notebook 2 and it analyzes:

- the distance between different feature sets in the parameter space
- the distance between different feature sets in the feature space
- the distance between different feature sets in the extracellular signals

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil

import bluepyopt as bpopt
import bluepyopt.ephys as ephys
import neuroplotlib as nplt

import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path
from pprint import pprint

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
# general
base_folder =Path("../../..")
cell_name = "cell1_211006_3148"
model_name = cell_name
ephys_dir = base_folder / "experimental_data" / cell_name / "patch_data"

cell_models_folder = base_folder/ "cell_models"
model_folder = cell_models_folder / cell_name

In [ ]:
result_folder = base_folder / "results" / '211124'

In [ ]:
save_fig = True
figure_folder = Path(".") / f"figures_{cell_name}"

if save_fig:
    figure_folder.mkdir(exist_ok=True)

# Define cell model and load optimization output

In [ ]:
cell = mf.create_experimental_model(model_name=cell_name)
cell_sample = mf.create_experimental_model(model_name=cell_name, release=True)

probe = mf.define_electrode(probe_file=model_folder / "fitting" / "efeatures" / "probe_BPO.json")
param_names = [param.name for param in cell.params.values() if not param.frozen]

params_sample = {}
for param in cell_sample.params_by_names(param_names):
    params_sample[param.name] = param.value

In [ ]:
pkl_file_name = "runs_experimental.pkl"

In [ ]:
data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == 'experimental'")

In [ ]:
opt_soma = df_model.query("feature_set == 'soma'")
opt_extra = df_model.query("feature_set == 'extra'")
print(f"Somatic optimizations: {len(opt_soma)}")
print(f"Extra optimizations: {len(opt_extra)}")

In [ ]:
fig, ax = plt.subplots()
min_evals = 3000
color_strategy = {"all": "C1", "sections": "C2", "single": "C3"}

keep_idxs_soma = []
for idx, row in opt_soma.iterrows():
    if max(row["nevals"]) > min_evals:
        keep_idxs_soma.append(idx)
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color="C0",
                ls='-', 
                lw=0.8,
                alpha=0.75)
    else:
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color="C0",
                ls='--', 
                lw=0.5,
                alpha=0.75)
keep_idx_extra = []
for idx, row in opt_extra.iterrows():
    #print(max(row['nevals']))
    if max(row["nevals"]) > min_evals:
        keep_idx_extra.append(idx)
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color=color_strategy[row["extra_strategy"]],
                ls='-', 
                lw=0.8,
                alpha=0.75, 
                label=idx)
    else:
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color=color_strategy[row["extra_strategy"]],
                ls='--', 
                lw=0.5,
                alpha=0.75, 
                label=idx)
ax.set_title("Min fitness")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xlabel("Neval")
ax.set_ylabel("Min fitness")
ax.set_yscale('log')

# Load protocols and experimental features

In [ ]:
# protocols_file = data_folder / "efeatures" / "protocols_BPO_all.json"
# feature_file = data_folder / "efeatures" / "features_BPO_all.json"
# all_protocols_file = data_folder / "efeatures" / "protocols.json"
# all_feature_file = data_folder / "efeatures" / "features.json"
# probe_file = data_folder / "efeatures" / "probe_BPO.json"

In [ ]:
extra_kwargs = mf.utils.get_extra_kwargs()
extra_kwargs["ms_cut"] = [2, 5]

In [ ]:
protocol_for_eap = "IDrest_300"

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    extra_strategy="all",
    protocols_with_lfp=protocol_for_eap,
    **extra_kwargs
)

## Load experimental responses

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    group_efeatures, create_feature_protocol_files, convert_legacy_targets
from bluepyefe.plotting import plot_all_recordings_efeatures

from multimodalfitting.efeatures_extraction import build_wcp_metadata, wcp_reader, get_ecode_targets, \
    ecodes_wcp_timings

In [ ]:
# define timings for this experiment
ecodes_cell_timings = {
    "IDthres": {
        'ton': 400,
        'toff': 670
    },
    "firepattern": {
        'ton': 500,
        'toff': 4100
    },
    "IV": {
        'ton': 400,
        'toff': 3400
    },
    "IDrest": {
        'ton': 400,
        'toff': 1750
    },
    "APWaveform": {
        'ton': 350,
        'toff': 400
    },
    "HyperDepol": {
        'ton': 400,
        'toff': 1120,
        'tmid': 850
    },
    "sAHP": {
        'ton': 400,
        'toff': 1325,
        'tmid': 650,
        'tmid2': 875
    },
    "PosCheops": {
        'ton': 1000,
        't1': 9000,
        't2': 10500,
        't3': 14500,
        't4': 16000,
        'toff': 18660
    }
}

In [ ]:
runs = [1, 2, 3, 4, 5]  # run1 --> different rheobase

ecode_names = list(ecodes_cell_timings.keys())

files_list = []

for run in runs:
    rep_dict = {}
    for ecode in ecode_names:
        for patch_file in ephys_dir.iterdir():
            if f"run{run}" in patch_file.name and ecode.lower() in patch_file.name:
                rep_dict[ecode] = patch_file
    files_list.append(rep_dict)

In [ ]:
files_metadata = build_wcp_metadata(cell_id=cell_name, 
                                    files_list=files_list, 
                                    ecode_timings=ecodes_cell_timings, 
                                    repetition_as_different_cells=False)
pprint(files_metadata[cell_name])

In [ ]:
cells = read_recordings(
    files_metadata=files_metadata,
    recording_reader=wcp_reader
)

In [ ]:
# define target features for different protocols
targets_legacy = get_ecode_targets(ecodes_cell_timings)

In [ ]:
global_tolerance = 30
for protocol, prot_dict in targets_legacy.items():
    prot_dict["tolerances"] = [global_tolerance]

In [ ]:
targets = convert_legacy_targets(targets_legacy)

In [ ]:
pprint(targets)

In [ ]:
compute_rheobase?

In [ ]:
compute_rheobase(
    cells, 
    protocols_rheobase=['IDthres'],
    rheobase_strategy="majority",
    majority=0.6
)

In [ ]:
print(f"Cell rheobase: {cells[0].rheobase}")

In [ ]:
protocols = group_efeatures(cells, targets, use_global_rheobase=True)

In [ ]:
protocols_opt = list(eva_extra.fitness_protocols.keys())

### Build BPO response dicts

In [ ]:
responses_experimental = []
num_runs = len(protocols[0].recordings)
for run in range(num_runs):
    print(f"Populating responses for run {run}")
    response_dict = {}
    for protocol in protocols:
        for i, rec in enumerate(protocol.recordings):
            if i == run:
                if protocol.stimulus_name in protocols_opt:
                    response = bpopt.ephys.responses.TimeVoltageResponse(name=protocol.name,
                                                                         time=rec.t, voltage=rec.voltage)
                    response_dict[f"{protocol.stimulus_name}.soma.v"] = response
    responses_experimental.append(response_dict)

In [ ]:
responses_experimental

In [ ]:
fig_exp_intra = mf.plot_multiple_responses(responses_experimental, return_fig=True, 
                                           labels=["run1", "run2", "run3", "run4", "run5"])

In [ ]:
eap_exp = np.load(model_folder / "fitting" / "efeatures" / "template_BPO.npy") / 1000

In [ ]:
ax = mu.plot_mea_recording(eap_exp, probe)
mf.plot_cell(eva_extra.cell_model, sim=eva_extra.sim, param_values=params_sample, 
             ax=ax, color_ais="r", color_axon="y")
fig_exp_extra = ax.get_figure()

In [ ]:
if save_fig:
    fig_exp_intra.savefig(figure_folder / "exp_intra.png", dpi=300)
    fig_exp_extra.savefig(figure_folder / "exp_extra.png", dpi=300)

## "soma" example

In [ ]:
best_soma = np.argmin(opt_soma.best_fitness)

In [ ]:
params_sample_soma = opt_soma.iloc[best_soma]
params_sample_soma_dict = {k: v for k, v in zip(param_names, params_sample_soma.best_params)}
print(params_sample_soma.best_fitness)

In [ ]:
params_sample_soma_dict

In [ ]:
response_soma = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), param_values=params_sample_soma_dict)

In [ ]:
eap_soma = mf.utils.calculate_eap(responses=response_soma, protocols=eva_extra.fitness_protocols, 
                                  protocol_name=protocol_for_eap, **extra_kwargs)

In [ ]:
fig_soma = mf.plot_multiple_responses([responses_experimental[1], response_soma], colors=["k", "C0"],
                                      return_fig=True, labels=["EXP", "SOMA"])

In [ ]:
if save_fig:
    fig_soma.savefig(figure_folder / "soma_intra.png", dpi=300)

In [ ]:
# plot extracellular
vscale = 1 * np.max(np.abs(eap_exp))
ax_soma = mu.plot_mea_recording(eap_exp, probe, vscale=vscale, lw=1)
ax_soma = mu.plot_mea_recording(eap_soma, probe, ax=ax_soma, vscale=vscale, colors="C0", lw=1)

In [ ]:
eap_dist_soma = distance.cosine(eap_exp.ravel(), eap_soma.ravel())
print(eap_dist_soma)

## "extra" example

In [ ]:
opt_all = opt_extra.query("extra_strategy == 'all'")
opt_sections = opt_extra.query("extra_strategy == 'sections'")
opt_single = opt_extra.query("extra_strategy == 'single'")

In [ ]:
best_all = np.argmin(opt_all.best_fitness)
best_sections = np.argmin(opt_sections.best_fitness)
best_single = np.argmin(opt_single.best_fitness)

In [ ]:
params_sample_all = opt_all.iloc[best_all]
params_sample_dict_all = {k: v for k, v in zip(param_names, params_sample_all.best_params)}
display("ALL", params_sample_all.best_fitness)

params_sample_sections = opt_sections.iloc[best_sections]
params_sample_dict_sections = {k: v for k, v in zip(param_names, params_sample_sections.best_params)}
display("SECTIONS", params_sample_sections.best_fitness)

params_sample_single = opt_single.iloc[best_single]
params_sample_dict_single = {k: v for k, v in zip(param_names, params_sample_single.best_params)}
display("SINGLE", params_sample_single.best_fitness)

In [ ]:
response_all = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                       param_values=params_sample_dict_all)
response_sections = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                            param_values=params_sample_dict_sections)
response_single = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                          param_values=params_sample_dict_single)

In [ ]:
eap_all = mf.utils.calculate_eap(responses=response_all, protocols=eva_extra.fitness_protocols, 
                                 protocol_name=protocol_for_eap, **extra_kwargs)
eap_sections = mf.utils.calculate_eap(responses=response_sections, protocols=eva_extra.fitness_protocols, 
                                      protocol_name=protocol_for_eap, **extra_kwargs)
eap_single = mf.utils.calculate_eap(responses=response_single, protocols=eva_extra.fitness_protocols, 
                                    protocol_name=protocol_for_eap, **extra_kwargs)

In [ ]:
mf.plot_multiple_responses([responses_experimental[1], response_all, 
                            response_sections, response_single], 
                           colors=["k", "C1", "C2", "C3"])

In [ ]:
# plot extracellular
fig, ax_extra = plt.subplots(figsize=(10, 7))
vscale = 1 * np.max(np.abs(eap_exp))
ax_extra = mu.plot_mea_recording(eap_exp, probe, vscale=vscale, lw=2, ax=ax_extra)
# ax_extra = mu.plot_mea_recording(eap_all, probe, ax=ax_extra, vscale=vscale, 
#                                  colors="C1", lw=1)
ax_extra.get_lines()[-1].set_label("EXP")
ax_extra = mu.plot_mea_recording(eap_sections, probe, ax=ax_extra, vscale=vscale, 
                                 colors="C2", lw=2)
ax_extra.get_lines()[-1].set_label("SECTIONS")
ax_extra.legend()

mf.plot_cell(eva_extra.cell_model, sim=eva_extra.sim, param_values=params_sample_dict_all, 
             ax=ax_extra, color_ais="r", color_axon="y")
# ax_extra = mu.plot_mea_recording(eap_single, probe, ax=ax_extra, vscale=vscale, 
#                                  colors="C3", lw=1)

In [ ]:
if save_fig:
    fig.savefig(figure_folder / "sections_extra.png", dpi=300)    

In [ ]:
eap_dist_all = distance.cosine(eap_exp.ravel(), eap_all.ravel())
eap_dist_sections = distance.cosine(eap_exp.ravel(), eap_sections.ravel())
eap_dist_single = distance.cosine(eap_exp.ravel(), eap_single.ravel())

In [ ]:
print("Cosine dist SOMA", eap_dist_soma)
print("Cosine dist EXTRA ALL", eap_dist_all)
print("Cosine dist EXTRA SECTIONS", eap_dist_sections)
print("Cosine dist EXTRA SINGLE", eap_dist_single)

## Run protocols and compute features for all tested models


In [ ]:
feats_soma = []
eaps_soma = []
params_soma = []
responses_soma = []

print("SOMA")
for i, (index, opt) in enumerate(opt_soma.iterrows()):
    print(f"Computing {i + 1} / {len(opt_soma)}")
    # get best params
    params_dict = {k: v for k, v in zip(param_names, opt.best_params)}
    params_soma.append(params_dict)
    
    # run protocols
    responses = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                        param_values=params_dict) 
    responses_soma.append(responses)
    try:
        eap = mf.utils.calculate_eap(responses=responses, protocols=eva_extra.fitness_protocols, 
                                  protocol_name=protocol_for_eap, **extra_kwargs)
    except:
        eap = np.zeros(eaps_soma[-1].shape)
    feat_dict = {}
    for obj in eva_extra.fitness_calculator.objectives:
        feat_dict[obj.features[0].name] = {}
        if len(obj.features) == 1:
            feat_value = obj.features[0].calculate_feature(responses)
            feat_score = obj.features[0].calculate_score(responses)
            feat_dict[obj.features[0].name]["value"] = feat_value
            feat_dict[obj.features[0].name]["score"] = feat_score
        else:
            print(f"More than one feature for objective: {obj.name}")
            
    feats_soma.append(feat_dict)
    eaps_soma.append(eap)
    
print("EXTRA")
feats_extra = []
eaps_extra = []
params_extra = []
responses_extra = []

for i, (index, opt) in enumerate(opt_extra.iterrows()):
    print(f"Computing {i + 1} / {len(opt_soma)}")
    # get best params
    params_dict = {k: v for k, v in zip(param_names, opt.best_params)}
    params_extra.append(params_dict)
    
    # run protocols
    responses = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                        param_values=params_dict)  
    responses_extra.append(responses)
    try:
        eap = mf.utils.calculate_eap(responses=responses, protocols=eva_extra.fitness_protocols, 
                                  protocol_name=protocol_for_eap, **extra_kwargs)
    except:
        eap = np.zeros(eaps_extra[-1].shape)
        
    feat_dict = {}
    for obj in eva_extra.fitness_calculator.objectives:
        feat_dict[obj.features[0].name] = {}
        if len(obj.features) == 1:
            feat_value = obj.features[0].calculate_feature(responses)
            feat_score = obj.features[0].calculate_score(responses)
            feat_dict[obj.features[0].name]["value"] = feat_value
            feat_dict[obj.features[0].name]["score"] = feat_score
        else:
            print(f"More than one feature for objective: {obj.name}")
            
    feats_extra.append(feat_dict)
    eaps_extra.append(eap)

In [ ]:
feature_set_array = []
seed_array = []
feature_name_array = []
feature_value_array = []
feature_score_array = []
feature_type_array = []

for i, feats in enumerate(feats_soma):
    for feat_name, feat_dict in feats.items():
        feature_set_array.append("soma")
        seed_array.append(i)
        feature_name_array.append(feat_name)
        if "MEA" not in feat_name:
            feature_type_array.append("soma")
            feature_value_array.append(feat_dict["value"])
        else:
            feature_type_array.append("MEA")
            feature_value_array.append(np.nan)
        feature_score_array.append(feat_dict["score"])
    
for i, feats in enumerate(feats_extra):
    for feat_name, feat_dict in feats.items():
        feature_set_array.append("extra")
        seed_array.append(i)
        feature_name_array.append(feat_name)
        if "MEA" not in feat_name:
            feature_type_array.append("soma")
            feature_value_array.append(feat_dict["value"])
        else:
            feature_type_array.append("MEA")
            feature_value_array.append(np.nan)
        feature_score_array.append(feat_dict["score"])
    
df_feats = pd.DataFrame({"seed": seed_array, "feature_set": feature_set_array, "feat_name": feature_name_array,
                         "feat_value": feature_value_array, "feat_score": feature_score_array,
                         "feat_type": feature_type_array})

In [ ]:
df_feats

In [ ]:
plt.figure()
sns.barplot(data=df_feats, x="feature_set", y="feat_score")

In [ ]:
fig = plt.figure()
ax = sns.barplot(data=df_feats.query("feat_type == 'MEA'"), 
            y="feat_name", x="feat_score", hue="feature_set")
sns.despine(left=True, bottom=True)
fig.subplots_adjust(left=0.4)

In [ ]:
fig = plt.figure()
sns.barplot(data=df_feats.query("feat_type == 'soma'"), 
            y="feat_name", x="feat_score", hue="feature_set")
sns.despine(left=True, bottom=True)
fig.subplots_adjust(left=0.4)

## Compare EAP distances

In [ ]:
feature_set_array = []
seed_array = []
eap_dist_array = []

for i, eap in enumerate(eaps_soma):
    feature_set_array.append("soma")
    seed_array.append(i)
    eap_dist_array.append(distance.cosine(eap_exp.ravel(), eap.ravel()))
    
for i, eap in enumerate(eaps_extra):
    feature_set_array.append("extra")
    seed_array.append(i)
    eap_dist_array.append(distance.cosine(eap_exp.ravel(), eap.ravel()))

    
df_eap = pd.DataFrame({"seed": seed_array, "feature_set": feature_set_array, "eap_dist": eap_dist_array})

In [ ]:
fig, ax = plt.subplots()
sns.barplot(data=df_eap, x="feature_set", y="eap_dist", ax=ax)
sns.swarmplot(data=df_eap, x="feature_set", y="eap_dist", dodge=True, ax=ax, palette=["cornflowerblue", "orange"])

# Plot responses

In [ ]:
fig_soma = mf.plot_multiple_responses(responses_list=responses_soma + responses_experimental, 
                                            max_rows=3, colors=["C0"] * len(responses_soma) + ['k'] * 
                                            len(responses_experimental), 
                                            return_fig=True)

In [ ]:
fig_extra = mf.plot_multiple_responses(responses_list=responses_extra + responses_experimental, 
                                             max_rows=3, colors=["C1"] * len(responses_extra) + ['k'] * 
                                             len(responses_experimental), 
                                             return_fig=True)